In [1]:
from PIL import Image,ImageDraw
import uuid
import os
import re
import random
import math

def calculate_distance(x1, y1, x2, y2):
    """二点間の距離を計算する"""
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def find_new_circle_coordinates(x, y, r, r2):
    """条件を満たす新しい円の座標を見つける"""
    while True:
        # 新しい円の中心座標を乱数で生成
        x2 = random.randint(0, 224)
        y2 = random.randint(0, 224)

        # 二つの円の中心間の距離を計算
        distance = calculate_distance(x, y, x2, y2)

        # 条件を満たすか確認
        if r - r2 < distance < r + r2:
            return x2, y2

# 既存の円の中心座標と半径
x, y, r = random.randint(0, 224), random.randint(0, 224), random.randint(0, 224)

# 新しい円の半径
r2 = random.randint(0, 224)

# 新しい円の中心座標を見つける
x2, y2 = find_new_circle_coordinates(x, y, r, r2)

(x, y, r), (x2, y2, r2)


def paste_transparent_image_centered(background_image_path, transparent_image_path, center_position, rate, R, image):
     # 背景画像を読み込む
    if image != '':
        background = image
        
    else:
        background = Image.open(background_image_path)
        

    # 透過画像を読み込む
    transparent = Image.open(transparent_image_path)
    
    # 透過画像をリサイズ
    original_size = transparent.size
    new_size = (int(original_size[0] * rate), int(original_size[1] * rate))
    transparent = transparent.resize(new_size, Image.LANCZOS)


    # 透過画像のサイズを取得
    tr_width, tr_height = transparent.size

    # 透過画像を中心座標に基づいて貼り付けるための位置を計算
    x = center_position[0] - tr_width // 2
    y = center_position[1] - tr_height // 2

    
    # 透過画像を貼り付け
    background.paste(transparent, (x, y), transparent)
    
    # 出力ファイル名を生成（座標とUUIDを含む）
    output_filename = f'{rate}_{center_position[0]}_{center_position[1]}_{str(int(rate*R))}_{uuid.uuid4()}.png'


    # 結果を保存
    # background.save(output_path+output_filename)
    return background, output_filename

def is_yellow(pixel, threshold=100):
    """ピクセルが黄色かどうかを判断する。"""
    r, g, b = pixel
    return r > threshold and g > threshold and b < threshold

def add_noise_to_yellow(image, noise_level=100, color=(0, 0, 0)):
    """黄色の部分にノイズを追加する関数。"""
    draw = ImageDraw.Draw(image)
    width, height = image.size

    for _ in range(noise_level):
        # ランダムな位置を選ぶ
        x = random.randint(0, width - 1)
        y = random.randint(0, height - 1)
        # その位置が黄色い場合、ノイズを追加
        if is_yellow(image.getpixel((x, y))):
            draw.point((x, y), fill=color)

    return image


In [2]:

os.makedirs("image/dataset/paste/simple", exist_ok=True)
os.makedirs("image/dataset/paste/double", exist_ok=True)

## データ生成

In [5]:
data_path='image/background/'
sponge_path = 'image/sponge/'
output_path = 'image/dataset/paste/'

entries = os.listdir(data_path)

# Filter to include only .jpg .png files
img_files = [file for file in entries if file.endswith(('.jpg','.png', 'jpeg'))]

sponge_entries = os.listdir(sponge_path)
# Filter to include only .jpg .png files
sponge_files = [file for file in sponge_entries if file.endswith(('.jpg','.png'))]
print(img_files)
print(sponge_files)

['background6.jpg', 'background2.jpg', 'background4.jpg', 'background3.jpeg', 'background1.jpeg', 'background_image_0.png', 'background5.jpg']
['60_sponge.png', '35_sponge_lack1.png', '88_sponge_v3.png', '23_sponge_lack3.png', '45_sponge_with_noise.png', '36_sponge_lack2.png', '60_sponge_shadow.png', '88_sponge_v3_shadow.png']


## 黒背景にスポンジボールを一個貼り付ける

In [6]:
#生成枚数
N = 225*225
output_path = 'image/dataset/paste/simple/'
#rate 100
#position 225*225
for i in range(N):
    global img_files
    background_image_path =  data_path + random.choice(img_files)
    transparent_image_path = sponge_path + random.choice(sponge_files)
    rate = random.uniform(0.1, 1.0)
    rate = round(rate, 2)
    center_position = (random.randint(0, 224 ), random.randint(0, 224))
    
    # 画像名から座標を抽出
    coords = re.findall(r"(\d+\.\d+|\d+)", transparent_image_path)
    R = int(coords[0])
    
    img, filename = paste_transparent_image_centered(background_image_path, transparent_image_path ,center_position, rate, R,'')
    img.save(output_path+filename)

## マルチコア　

In [ ]:
import random
import re
from PIL import Image
import concurrent.futures

def generate_and_save_image(image_id, data_path, sponge_path, img_files, sponge_files, output_path):
    background_image_path = data_path + random.choice(img_files)
    transparent_image_path = sponge_path + random.choice(sponge_files)
    rate = round(random.uniform(0.1, 1.0), 2)
    center_position = (random.randint(0, 224), random.randint(0, 224))

    coords = re.findall(r"(\d+\.\d+|\d+)", transparent_image_path)
    R = int(coords[0])

    img, filename = paste_transparent_image_centered(background_image_path, transparent_image_path, center_position, rate, R, '')
    img.save(output_path + filename)

    return f'Image {image_id} created'

def create_images_concurrently(num_images, data_path, sponge_path, img_files, sponge_files, output_path):
    with concurrent.futures.ProcessPoolExecutor(max_workers=5) as executor:
        # 各画像に対してgenerate_and_save_image関数を並列実行
        futures = [executor.submit(generate_and_save_image, i, data_path, sponge_path, img_files, sponge_files, output_path) for i in range(num_images)]
        # 結果を取得
        results = [future.result() for future in concurrent.futures.as_completed(futures)]

    return results

# 例：75,000個の画像を並列で生成
num_images = 25000
results = create_images_concurrently(num_images, data_path, sponge_path, img_files, sponge_files, output_path)



## スポンジボールを二個貼り付ける。（大きい方の座標を取得）

In [7]:
#生成枚数
N = 1
output_path = 'image/dataset/paste/double/'
#rate 100
#position 225*225
for i in range(N):
    global img_files
    background_image_path =  data_path + random.choice(img_files)
    transparent_image_path = sponge_path + random.choice(sponge_files)
    rate = random.uniform(0.1, 1.0)
    rate = round(rate, 2)
    center_position = (random.randint(0, 224 ), random.randint(0, 224))
    
    # 画像名から座標を抽出
    coords = re.findall(r"(\d+\.\d+|\d+)", transparent_image_path)
    R = int(coords[0])
    
    img, filename1 = paste_transparent_image_centered(background_image_path, transparent_image_path ,center_position, rate, R,'')

    # 画像名から座標を抽出
    coords = re.findall(r"(\d+\.\d+|\d+)", filename1)
    pre_rate =  float(coords[0])
    x, y, r = int(coords[1]), int(coords[2]) , int(coords[3])  
    if pre_rate > 0.5:
    # rate > 0.5 の場合、0.1からrateまでの乱数を生成
        new_rate = random.uniform(0.1, pre_rate/1.5)
        save_file = 1
    else:
    # rate < 0.5 の場合、rateから1までの乱数を生成
        new_rate = random.uniform(pre_rate*1.5, 1.0)
        save_file = 2
        
    new_rate = round(new_rate, 2)
    # 新しい円の半径
    R2 = int(new_rate*R)
    # 新しい円の中心座標を見つける
    center_position = find_new_circle_coordinates(x, y, r, R2)
    
    img, filename2 = paste_transparent_image_centered(background_image_path, transparent_image_path ,center_position, new_rate, R, img)
    
    if save_file == 1:
        img.save(output_path+filename1)
    elif save_file == 2:
        img.save(output_path+filename2)
    

    
    

## マルチコア

In [ ]:
import concurrent.futures
import random
import re
from PIL import Image

def generate_double_image(image_id, data_path, sponge_path, img_files, sponge_files, output_path):
    background_image_path = data_path + random.choice(img_files)
    transparent_image_path = sponge_path + random.choice(sponge_files)
    rate = round(random.uniform(0.1, 1.0), 2)
    center_position = (random.randint(0, 224), random.randint(0, 224))

    # 画像名から座標を抽出してRを取得
    coords = re.findall(r"(\d+\.\d+|\d+)", transparent_image_path)
    R = int(coords[0])

    img, filename1 = paste_transparent_image_centered(background_image_path, transparent_image_path, center_position, rate, R, '')

    coords = re.findall(r"(\d+\.\d+|\d+)", filename1)
    pre_rate = float(coords[0])
    x, y, r = int(coords[1]), int(coords[2]), int(coords[3])

    # 新しいレートを決定
    if pre_rate > 0.5:
        new_rate = random.uniform(0.1, pre_rate / 1.5)
        save_file = 1
    else:
        new_rate = random.uniform(pre_rate * 1.5, 1.0)
        save_file = 2

    new_rate = round(new_rate, 2)
    R2 = int(new_rate * R)
    center_position = find_further_circle_coordinates(x, y, r, R2)

    img, filename2 = paste_transparent_image_centered(background_image_path, transparent_image_path, center_position, new_rate, R, img)

    if save_file == 1:
        img.save(output_path + filename1)
    elif save_file == 2:
        img.save(output_path + filename2)

    return f'Image {image_id} created'


In [ ]:
def create_double_images_concurrently(num_images, data_path, sponge_path, img_files, sponge_files, output_path):
    with concurrent.futures.ProcessPoolExecutor(max_workers=16) as executor:
        futures = [executor.submit(generate_double_image, i, data_path, sponge_path, img_files, sponge_files, output_path) for i in range(num_images)]
        results = [future.result() for future in concurrent.futures.as_completed(futures)]

    return results



# 例：50,000個の画像を並列で生成
num_images = 20000
output_path = 'image/dataset/paste/double/'
results = create_double_images_concurrently(num_images, data_path, sponge_path, img_files, sponge_files, output_path)
print(results)
